In [13]:
import networkx
import math
import random
import copy
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm 
import pickle
import sys 

In [14]:
Graph_2 = networkx.read_gpickle("Graph2")

In [15]:
# print(Graph_2.number_of_nodes())
# print(Graph_2.number_of_edges())
list_edges = list(Graph_2.edges())
list_nodes = list(Graph_2.nodes())
value_c = 5

In [16]:
dict_dependency = {}
dict_bfs_tree = {}
dict_distances = {}
dict_shortest_path_counts = {}
dict_predecessors = {}
dict_dict_parents = {}
count_nodes = Graph_2.number_of_nodes()

In [17]:
dict_edge_betweenness = {}
for ede in list_edges:
    dict_edge_betweenness[ede] = 0
edge_list = Graph_2.edges()
counter_edges = Counter(edge_list)

In [18]:
def find_dependency(src, edg):
    global dict_dependency
    u = edg[0]
    v = edg[1]
    distance_to_u = dict_distances[src][u]
    distance_to_v = dict_distances[src][v]
    if distance_to_u > distance_to_v:
        temp_store = u
        u = v
        v = temp_store
    value_dependency = dict_shortest_path_counts[src][u] / dict_shortest_path_counts[src][v]
    if current_node_bfs_tree.out_degree(v) == 0:
        if counter_edges[edg] == 1:
            dict_dependency[src][edg] = value_dependency
            return value_dependency
        else:    
            return value_dependency

    else:
        #Now find vertices which containg vertex v in their parent set
        list_x_nodes = []
        dict_curr_parents = dict_dict_parents[src]
        for aq in dict_curr_parents:
            list_parents = dict_curr_parents[aq]
            if v in list_parents:
                list_x_nodes.append(aq)

        #Finding the value of summation
        value_summation = 0
        for x in list_x_nodes:
            req_ed = (v, x)
            if dict_dependency[src].get(req_ed) is None:
                value_summation += find_dependency(src, req_ed)
            else:    
                value_summation += dict_dependency[src][req_ed]
        final_val = value_dependency * (1 + value_summation)
        if counter_edges[edg] == 1:
            dict_dependency[src][edg] = final_val
        else:
            return final_val

def edge_betweenness(start_node):
    global dict_dependency
    dict_dependency[start_node] = {}
    global current_node_bfs_tree 
    current_node_bfs_tree = dict_bfs_tree[start_node]
    for e in list_edges:
        if dict_dependency[start_node].get(e) is None:
            dict_dependency[start_node][e] =  find_dependency(start_node, e)

In [19]:
len_list_nodes = len(list_nodes)
#First initilaizing the dictionaries for different purpose
for a in list_nodes:
    dict_bfs_tree[a] = []
    dict_distances[a] = []
    dict_shortest_path_counts[a] = []
    dict_predecessors[a] = []

for a in list_nodes:
    dict_dict_parents[a] = copy.deepcopy(dict_predecessors)
    
val_infinity = math.inf
for i in range(len_list_nodes):
    curr_node = list_nodes[i]
    temp = [val_infinity for j in range(len_list_nodes)]
    dict_distances[curr_node] = temp
    dict_distances[curr_node][curr_node] = 0
    dict_shortest_path_counts[curr_node] = [0 for k in range(len_list_nodes)]
    dict_shortest_path_counts[curr_node][curr_node] = 1
    
k_val = 0
while k_val < int(count_nodes / 10):
    k_val += 1
    sample_node = random.randint(0, 999)
    Graph_bfs = networkx.DiGraph()
    v = copy.deepcopy(sample_node)
    
    list_visited = [0 for i in range(len_list_nodes)]
    bfs_queue = []
    list_visited[v] = 1
    bfs_queue.append(v)
    
    list_bfs_tree = []
    dict_prev = {}
    list_source_distances = dict_distances[v]
    list_source_counts = dict_shortest_path_counts[v]
    while len(bfs_queue) > 0:
        curr_node = bfs_queue.pop(0)
        if curr_node != v:
            Graph_bfs.add_edge(dict_prev[curr_node], curr_node)
        list_neighbours = [x for x in Graph_2.neighbors(curr_node)]
        for nd in list_neighbours:
            if list_visited[nd] == 0:
                list_visited[nd] = 1
                bfs_queue.append(nd)
                dict_prev[nd] = curr_node
            
            if list_source_distances[nd] > list_source_distances[curr_node] + 1:
                list_source_distances[nd] = list_source_distances[curr_node] + 1
                list_source_counts[nd] = list_source_counts[curr_node]
                dict_dict_parents[v][nd] = [curr_node]
            
            elif list_source_distances[nd] == list_source_distances[curr_node] + 1:
                list_source_counts[nd] = list_source_counts[nd] + list_source_counts[curr_node]
                dict_dict_parents[v][nd].append(curr_node)

    dict_distances[v] = list_source_distances
    dict_shortest_path_counts[v] = list_source_counts
    dict_bfs_tree[v] = Graph_bfs
    for b in dict_dict_parents[v]:
        temp_list = dict_dict_parents[v][b]
        x = set(temp_list)
        list_set = copy.deepcopy(list(x))
        flag = 0
        for i in range(len(list_set)):
            if list_set[i] == b:
                flag = 1
                break
        if flag == 1:
            del list_set[i]
        dict_dict_parents[a][b] = copy.deepcopy(list_set)
    
    #Calling the function which is preparing the dictionary containing the betweenness values of all edges for vertex v
    edge_betweenness(v)
    
    #Maintaing the running sum delta e in the dictionary dict_edge_betweenness
    for e in list_edges:
        dict_edge_betweenness[e] += dict_dependency[v][e]
    
    #Now checking if all delta e values are greater than cn
    global flag_sampling
    flag_sampling = 0
    for edge in dict_edge_betweenness:
        if dict_edge_betweenness[edge] <= (value_c * count_nodes):
            flag_sampling = 1
            break
    
    if flag_sampling == 0:
        break
    

if flag_sampling == 0:
    print("Condition satisfied by delta e")
else:
    print("Maximum no. of samples drawn")

Maximum no. of samples drawn


In [20]:
for edg in dict_edge_betweenness:
    dict_edge_betweenness[edg] = (count_nodes / k_val) * dict_edge_betweenness[edg]

In [21]:
# print(dict_edge_betweenness)

In [22]:
Picklefile2 = open('dictionary_betweenness_algo2', 'wb')
pickle.dump(dict_edge_betweenness, Picklefile2)
Picklefile2.close()